In [1]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta
import urllib

# вытащить ДФы для одного, пары, тройки героев из сайта по дате
def get_df_avgElo_heroes(day_match, df):
       
    # создать ДФ для одного героя, пары и тройки
    df_one = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 1), :]  
    df_one = df_one.reset_index().drop('index', axis=1)
#     df_double = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 2), :]  
#     df_triple = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 3), :]  

    return df_one#, df_double, df_triple

# вытащить таблицу с elo героев для команды 
def get_df_AvgElo_heroes_teams(radiant_or_dire):
    # id команды из данных по матчу
    id_team = int(main[radiant_or_dire + '_team_id'][index])
    # дата матча
    date_match = date.fromtimestamp(main['start_time'][index])
    
    # создать ссылку с данными по Avg.Elo для двух предыдущих месяцев игры
    url = ('http://www.datdota.com/api/heroes/elo?teams={}&tier=1&valve-event=does-not-matter&threshold=5'.format(id_team) +
        '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' + 
        '&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either'+
        '&after=01%2F01%2F2011'+ 
        '&before={}%2F{}%2F{}'.format(date_match.day-1, date_match.month, date_match.year) + 
        '&duration=0%3B200&duration-value-from=0&duration-value-to=200')
    # выгрузить все с сайта и создать ДФ
    sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_url = pd.DataFrame(dat)
    
    # создать ДФ для соло героев за все время игр команды (сигнатурки)
    return get_df_avgElo_heroes(date_match, df_url)

# Подгрузка мейн ДФ с id матчей

In [3]:
# загрузить главный ДФ
main = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv', index_col=0)
main = main[main['match_id'] == 3274564960]
main

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
1007,3274564960,1498454706,3659536.0,13,3326875.0,20,False,111.0,47.0,99.0,...,164532005.0,121833213.0,152545459.0,127077041.0,100471531.0,21289303.0,89603649.0,84772440.0,85417034.0,The International 2017


In [4]:
# создать основной ДФ, где будут записаны все матчи (id матча и rda героя)
df_AvgElo_heroes = pd.DataFrame()

# создать список названий колонок всех героев
all_her = main.loc[:,'radiant_H1':'dire_H5'].columns
for index in main.index:
    # ДФ для соединения всех герове матча в один ДФ 
    df_heroe_match = pd.DataFrame()
    # дата матча
    date_match = date.fromtimestamp(main['start_time'][index])
    # предыдущий день
    date_match =  date_match - timedelta(1)
    # создать дату два месяца назад от даты матча
    two_month_ago = date_match - timedelta(60)
    # создать ссылку с данными по Avg.Elo для двух предыдущих месяцев игры
    url = ('http://www.datdota.com/api/heroes/elo?tier=1&valve-event=does-not-matter&threshold=10' +
        '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' + 
        '&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either'+
        '&after={}%2F{}%2F{}'.format(two_month_ago.day, two_month_ago.month, two_month_ago.year) + 
        '&before={}%2F{}%2F{}'.format(date_match.day, date_match.month, date_match.year) + 
        '&duration=0%3B200&duration-value-from=0&duration-value-to=200')
    # выгрузить все с сайта и создать ДФ
    sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_url = pd.DataFrame(dat)
    # создать ДФ для одного героя за 2 предыдущих месяца
    df_data_tabel_for_heroes = get_df_avgElo_heroes(date_match, df_url)
    
    for her in all_her:
        # вытащить id героя
        id_hero = main[her][index]

        # вытащить avg elo для данного героя
        avgElo_hero = df_data_tabel_for_heroes.loc[(index for index, x in enumerate(
                                            df_data_tabel_for_heroes['heroes']) if x == [id_hero]),'eloShift']
        
        # проверить есть ли герой
        try:
            avgElo_hero = float(avgElo_hero)
        except:
            avgElo_hero = 0
        # ДФ для avgELo героя по матчу
        df_heroe = pd.DataFrame([avgElo_hero], columns=[her + '_AvgElo'])
        df_heroe_match = pd.merge(df_heroe_match, df_heroe, left_index=True, right_index=True, how='outer')
    df_AvgElo_heroes = pd.concat([df_AvgElo_heroes, df_heroe_match])
df_AvgElo_heroes = df_AvgElo_heroes.reset_index().drop('index', axis=1)
df_AvgElo_heroes['match_id'] = main['match_id']

HTTPError: HTTP Error 407: Proxy Authentication Required

In [4]:
df_AvgElo_heroes#.to_csv('../tabel/table from Datdota/tabel with AvgElo heroes (meta).csv')


In [5]:
url

'http://www.datdota.com/api/heroes/elo?tier=1&valve-event=does-not-matter&threshold=10&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either&after=26%2F4%2F2017&before=25%2F6%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200'

# Вытащить как команды играют за все время (сигнатурки)

In [5]:
# создать основной ДФ, где будут записаны все матчи (id матча и rda героя)
df_AvgElo_heroes_teams = pd.DataFrame()

# создать список названий колонок  героев radiant
all_her_rad = main.loc[:,'radiant_H1':'radiant_H5'].columns
# создать список названий колонок  героев radiant
all_her_dir = main.loc[:,'dire_H1':'dire_H5'].columns

for index in main.index:
    # ДФ для соединения всех герове radiant & dire матча в один ДФ 
    df_heroe_match = pd.DataFrame()
    
    # ДЛЯ РЕДИАНТ
    # создать ДФ c avgElo для редиант по дате игры
    try: # если нету данных по команде то вернуть пустой ДФ
        df_data_tabel_radiant = get_df_AvgElo_heroes_teams('radiant')
    except:
        df_data_tabel_radiant = pd.DataFrame()
        
    for her in all_her_rad:
        # вытащить id героя
        id_hero = main[her][index]
        # проверить есть ли герой
        try:
             # вытащить avg elo для данного героя
            avgElo_hero = df_data_tabel_radiant.loc[(index for index, x in enumerate
                                                (df_data_tabel_radiant['heroes']) if x == [id_hero]),'eloShift']
            avgElo_hero = float(avgElo_hero)
        except:
            avgElo_hero = 0
        
        # ДФ для avgELo героя по матчу
        df_heroe = pd.DataFrame([avgElo_hero], columns=[her + '_AvgElo'])
        df_heroe_match = pd.merge(df_heroe_match, df_heroe, left_index=True, right_index=True, how='outer') 
    
    # ДЛЯ ДАЕР
    # создать ДФ c avgElo для dire по дате игры
    try: # если нету данных по команде то вернуть пустой ДФ
        df_data_tabel_dire = get_df_AvgElo_heroes_teams('dire')
    except:
        df_data_tabel_dire = pd.DataFrame()
        
    for her in all_her_dir:
        # вытащить id героя
        id_hero = main[her][index]

        # проверить есть ли герой
        try:
            # вытащить avg elo для данного героя
            avgElo_hero = df_data_tabel_dire.loc[(index for index, x in enumerate
                                                (df_data_tabel_dire['heroes']) if x == [id_hero]),'eloShift']
            avgElo_hero = float(avgElo_hero)
        except:
            avgElo_hero = 0

        # ДФ для avgELo героя по матчу
        df_heroe = pd.DataFrame([avgElo_hero], columns=[her + '_AvgElo'])
        df_heroe_match = pd.merge(df_heroe_match, df_heroe, left_index=True, right_index=True, how='outer') 
    df_AvgElo_heroes_teams = pd.concat([df_AvgElo_heroes_teams, df_heroe_match])
       
df_AvgElo_heroes_teams = df_AvgElo_heroes_teams.reset_index().drop('index', axis=1)
df_AvgElo_heroes_teams['match_id'] = main['match_id']

In [6]:
df_AvgElo_heroes_teams.to_csv('../tabel/table from Datdota/tabel with AvgElo heroes for teams (signatures).csv')

In [32]:
df_AvgElo_heroes_teams[:4]

,radiant_H1_AvgElo,radiant_H2_AvgElo,radiant_H3_AvgElo,radiant_H4_AvgElo,radiant_H5_AvgElo,dire_H1_AvgElo,dire_H2_AvgElo,dire_H3_AvgElo,dire_H4_AvgElo,dire_H5_AvgElo,match_id
0,-2.414744,-6.983857,4.652648,0.000000,7.534793,-0.372673,-3.672918,8.703789,0.000000,8.345258,2897064589
1,0.000000,0.000000,13.427776,3.431928,-10.422860,-2.844923,8.265573,10.193163,-13.318344,0.000000,2897074199
2,0.158690,NaN,1.973433,NaN,-5.380925,-5.978656,NaN,0.000000,0.000000,10.844549,2897155244
3,5.672286,0.000000,-10.505205,0.000000,10.764666,0.000000,11.239669,0.000000,0.000000,0.000000,2897190211


In [33]:
main.loc[:3, 'radiant_win':'dire_H5']

,radiant_win,radiant_H1,radiant_H2,radiant_H3,radiant_H4,radiant_H5,dire_H1,dire_H2,dire_H3,dire_H4,dire_H5
0,True,86.0,8.0,76.0,57.0,107.0,54.0,46.0,62.0,25.0,28.0
1,False,21.0,55.0,71.0,48.0,84.0,9.0,54.0,79.0,39.0,96.0
2,False,79.0,63.0,28.0,98.0,16.0,86.0,100.0,2.0,82.0,48.0
3,True,20.0,11.0,28.0,14.0,111.0,84.0,10.0,101.0,13.0,19.0


In [56]:
df_data_tabel_radiant.loc[(index for index, x in enumerate
                                                (df_data_tabel_radiant['heroes']) if x == [87]),'eloShift']

25    5.025534
Name: eloShift, dtype: float64

In [63]:
df_data_tabel_radiant[df_data_tabel_radiant['heroes'] == np.array(87)]

ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [78]:
df_data_tabel_radiant = df_data_tabel_radiant.reset_index().drop('index', axis=1)

AttributeError: 'function' object has no attribute 'reset_index'

In [75]:
df_data_tabel_radiant

<bound method DataFrame.reindex of     eloGames   eloShift heroes  loss  total  win
0         36   8.765621   [20]    13     36   23
1         28   5.424745   [30]    15     32   17
2         30   0.990263   [74]    16     31   15
3         26   1.973433   [28]    11     26   15
4         23   6.773046  [107]    10     24   14
5         22   8.488584   [29]     7     22   15
6         19  -7.547376    [8]    11     19    8
7         17   1.663469   [13]     9     18    9
8         16   6.783781    [9]     7     17   10
9         13  -3.718490   [18]     8     13    5
10        13  -1.218205   [65]     7     13    6
11        10  18.298670   [54]     4     12    8
12        12   1.589819  [111]     5     12    7
13        10   0.929275   [55]     7     12    5
14        12  -5.380925   [16]     6     12    6
15        12   0.136884   [72]     7     12    5
16        12   2.216788  [103]     4     12    8
17        11 -14.264234   [26]     9     11    2
19        10   1.125036  [106]    